In [ ]:
from common import *
from snobedo.snotel import CsvParser
from snobedo.lib.isnobal_helpers import day_filter

use_hvplot()

In [ ]:
client = start_cluster(5, 12)
client

# Precipitation comparison across Snotel, HRRR forecast hours 01 and 06, and SMRF

In [ ]:
year = 2018
water_year = f'wy{year}'

melt_start = pd.to_datetime(f'{year}-04-01')

In [ ]:
snotel_sites = SnotelLocations()
snotel_sites.load_from_json(SNOTEL_DIR / 'site-locations/snotel_sites_2x2.json')

## HRRR Pixel

In [ ]:
SAVE_DEST = "wy{year}/{site.name}/{site.name}-HRRR-wy{year}.zarr"

schofield_hrrr = xr.open_zarr(
    SNOTEL_DIR / SAVE_DEST.format(year=year, site=snotel_sites.Schofield)
)

butte_hrrr = xr.open_zarr(
    SNOTEL_DIR / SAVE_DEST.format(year=year, site=snotel_sites.Butte)
)

taylor_hrrr = xr.open_zarr(
    SNOTEL_DIR / SAVE_DEST.format(year=year, site=snotel_sites.Taylor)
)

## HRRR f06 Pixel

In [ ]:
SAVE_DEST = "wy{year}/{site.name}/{site.name}-HRRR-wy{year}_sfc06.zarr"

schofield_hrrr_06 = xr.open_zarr(
    SNOTEL_DIR / SAVE_DEST.format(year=year, site=snotel_sites.Schofield)
)

butte_hrrr_06 = xr.open_zarr(
    SNOTEL_DIR / SAVE_DEST.format(year=year, site=snotel_sites.Butte)
)

taylor_hrrr_06 = xr.open_zarr(
    SNOTEL_DIR / SAVE_DEST.format(year=year, site=snotel_sites.Taylor)
)

## SNOTEL

In [ ]:
schofield_snotel_csv = CsvParser.file(
    SNOTEL_DIR / water_year / f'Schofield/usda-csv/{year}-Schofield-Pass.csv'
)

butte_snotel_csv = CsvParser.file(
    SNOTEL_DIR / water_year / f'Butte/usda-csv/{year}-Butte.csv',
)

taylor_snotel_csv = CsvParser.file(
    SNOTEL_DIR / water_year / f'Taylor/usda-csv/{year}-Upper-Taylor.csv',
)

## iSnobal 

In [ ]:
wy_precip = xr.open_mfdataset(
    (SNOBAL_DIR / water_year / 'erw/*/precip.nc').as_posix(),
    preprocess=day_filter,
    parallel=True,
)

In [ ]:
butte_snobal = wy_precip.sel(
    x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat
).resample(**RESAMPLE_1_DAY_OPTS).sum().compute()

schofield_snobal = wy_precip.sel(
    x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat
).resample(**RESAMPLE_1_DAY_OPTS).sum().compute()

taylor_snobal = wy_precip.sel(
    x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat
).resample(**RESAMPLE_1_DAY_OPTS).sum().compute()

In [ ]:
plot_data = {
    'Butte': {
        'HRRR': butte_hrrr,
        'HRRR FC06': butte_hrrr_06,
        'Snobal': butte_snobal,
        'Snotel': butte_snotel_csv,
    },
    'Schofield Pass': {
        'HRRR': schofield_hrrr,
        'HRRR FC06': schofield_hrrr_06,
        'Snobal': schofield_snobal,
        'Snotel': schofield_snotel_csv,
    },
    'Upper Taylor': {
        'HRRR': taylor_hrrr,
        'HRRR FC06': taylor_hrrr_06,
        'Snobal': taylor_snobal,
        'Snotel': taylor_snotel_csv,
    },
}

In [ ]:
butte_hrrr_06.tp.mean(['x', 'y']).hvplot(label='FC 06', color='darkorchid', width=1280, height=640, title='Butte') * \
butte_snotel_csv['Precipitation (mm)'].plot(label='Snotel', color='darkgrey') * \
butte_hrrr.tp.mean(['x', 'y']).hvplot(label='FC 00', color='limegreen') * \
butte_snobal.precip.mean(['x', 'y']).hvplot(color='orange', label='Snobal', alpha=0.8)

In [ ]:
schofield_hrrr_06.tp.mean(['x', 'y']).hvplot(label='FC 06', color='darkorchid', width=1280, height=640, title='Schofield Pass') * \
schofield_snotel_csv['Precipitation (mm)'].plot(label='Snotel', color='darkgrey') * \
schofield_hrrr.tp.mean(['x', 'y']).hvplot(label='FC 00', color='limegreen') * \
schofield_snobal.precip.mean(['x', 'y']).hvplot(color='orange', label='Snobal', alpha=0.8)

In [ ]:
taylor_hrrr_06.tp.mean(['x', 'y']).hvplot(label='FC 06', color='darkorchid', width=1280, height=640, title='Upper Taylor') * \
taylor_snotel_csv['Precipitation (mm)'].plot(label='Snotel', color='darkgrey') * \
taylor_hrrr.tp.mean(['x', 'y']).hvplot(label='FC 00', color='limegreen') * \
taylor_snobal.precip.mean(['x', 'y']).hvplot(color='orange', label='Snobal', alpha=0.8)

### Stats 

In [ ]:
for site in plot_data.keys():
    snotel = plot_data[site]['Snotel']['Precipitation (mm)'].sum()
    snobal = plot_data[site]['Snobal'].precip.mean(['x', 'y']).sum().values
    hrrr = plot_data[site]['HRRR'].tp.sum().compute().values
    hrrr_06 = plot_data[site]['HRRR FC06'].tp.sum().compute().values
    
    print(site)
    print("  Snotel Total: {0} mm".format(snotel))
    print("  iSnobal: {:.2%}".format(snobal/snotel))
    print("  HRRR F00: {:.2%}".format(hrrr/snotel))
    print("  HRRR F06: {:.2%}".format(hrrr_06/snotel))